In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
jtplot.style(figsize=(24, 9))

# VWAP. Volume-Weighted Average Price

In [2]:
df = pd.read_csv('../../../data/order_book/BAC.csv',
                 index_col=0, parse_dates=True)
df.head()

,Bid,Ask,Last,Volume,Type
Time,,,,,
2023-03-08 15:06:21.582,32.85,32.86,32.85,2101.0,Sell
2023-03-08 15:06:21.582,32.85,32.86,32.85,100.0,Sell
2023-03-08 15:06:21.582,32.85,32.86,32.85,100.0,Sell
2023-03-08 15:06:21.582,32.85,32.86,32.85,100.0,Sell
2023-03-08 15:06:21.582,32.85,32.86,32.85,400.0,Sell


In [3]:
df.shape

(100000, 5)

In [4]:
df['Volume'].is_monotonic_increasing

False

In [5]:
df.dtypes

Bid       float64
Ask       float64
Last      float64
Volume    float64
Type       object
dtype: object

In [6]:
df.index.inferred_type

'datetime64'

In [7]:
df.groupby('Type').count()

,Bid,Ask,Last,Volume
Type,,,,
Buy,61655,61655,61655,61655
Buy/Sell,91,91,91,91
Sell,38254,38254,38254,38254


In [9]:
data = df[df['Type']=='Buy'].copy()

In [41]:
data.head()

,Bid,Ask,Last,Volume,Type
Time,,,,,
2023-03-08 15:06:21.945,32.84,32.85,32.85,200.0,Buy
2023-03-08 15:06:26.047,32.83,32.84,32.84,100.0,Buy
2023-03-08 15:06:26.872,32.83,32.84,32.84,100.0,Buy
2023-03-08 15:06:27.591,32.83,32.84,32.84,1601.0,Buy
2023-03-08 15:06:29.501,32.82,32.83,32.83,200.0,Buy


In [40]:
target_volume = 2 # lot
duration_sec = 30

start_time = data.loc['2023-03-08 15:06'].iloc[0].name
end_time = start_time + pd.Timedelta(seconds=30)

start_time, end_time

(Timestamp('2023-03-08 15:06:21.945000'),
 Timestamp('2023-03-08 15:06:51.945000'))

In [48]:
period_data = data.loc[
    (data.index>=start_time) &
    (data.index<=end_time)
].copy()

period_data.head()

,Bid,Ask,Last,Volume,Type
Time,,,,,
2023-03-08 15:06:21.945,32.84,32.85,32.85,200.0,Buy
2023-03-08 15:06:26.047,32.83,32.84,32.84,100.0,Buy
2023-03-08 15:06:26.872,32.83,32.84,32.84,100.0,Buy
2023-03-08 15:06:27.591,32.83,32.84,32.84,1601.0,Buy
2023-03-08 15:06:29.501,32.82,32.83,32.83,200.0,Buy


In [56]:
period_data['time_index'] = np.round((period_data.index-start_time).total_seconds(), 3)

period_data.head()

,Bid,Ask,Last,Volume,Type,time_index
Time,,,,,,
2023-03-08 15:06:21.945,32.84,32.85,32.85,200.0,Buy,0.000
2023-03-08 15:06:26.047,32.83,32.84,32.84,100.0,Buy,4.102
2023-03-08 15:06:26.872,32.83,32.84,32.84,100.0,Buy,4.927
2023-03-08 15:06:27.591,32.83,32.84,32.84,1601.0,Buy,5.646
2023-03-08 15:06:29.501,32.82,32.83,32.83,200.0,Buy,7.556
